In [1]:
import pandas as pd
import numpy as np
import random, string
import datetime
from datetime import date
from random import randrange
from datetime import timedelta
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")
import boto3
import os
import logging

In [2]:
#Create and configure logger
logging.basicConfig(filename="newfile.log",
                    format='%(asctime)s %(message)s',
                    filemode='w')
  
#Creating an object
logger=logging.getLogger()

logger.info("Notebook Execution Part 1 - Main Raw Material Table")

In [3]:
# bucket setup
s3 = boto3.resource('s3')
bucket_name = 'jd-processed-1'

In [4]:
bossard = pd.read_parquet("s3://reformatted-bossard/bossardjdcollab.parquet", engine = "pyarrow")
# bossard = pd.read_csv("File from the Supplier.csv")
deere = pd.read_csv("Bossard Extract_v2_final", engine = 'c')
# deere = pd.read_csv("File from Client.csv", engine = 'c')
commodity = pd.read_csv("CommodityPPI.csv").drop('DATE', axis = 1)
# deere = pd.read_excel("Bossard Extract_v2_final (1).xlsx", engine = "openpyxl")[1:]

In [5]:
columns = ['PO Number', 'Part Number', 'SAP Line Down Date', 'SAP Line Down Including ASN', 'Part on Waterloo Critical Report?', 'Waterloo Report DOH', '# of Units that Use this Part',
'Annual EAU per part # (All units)', 'On Hand Inventory at Deere', 'In Transit Inventory to Deere', 'Deere Ordering Location Stock + ASN Qty',
'Supplier Stock', 'P/N: Per Unit Price (Deere Data/Supplier Data)', 'Base Commodity', 'Base Commodity PPI',
'Base Commodity PPI Change', 'Base Commodity Change%', 'Supplier Critical Part Number',
'Supplier Can Build Critical Part Qty: On Hand', 'Tier N Supplier PO#', 'Tier N Supplier PO Due Date', 'Tier N Supplier PO Forecast Date',
'Tier N PO Qty', 'Tier N Supplier Location (Country)', 'Supplier Tier N Supplier Name', 'Critical Part Alert', 'Deere PO Demand',
'Is Supplier Tier N Forecast Date within 30 days of Deere Need Date? (0 - No, 1 - Yes)', 'Does Inventory On-Hand Meet Demand including able to build (0 - No, 1 - Yes)',
'Inventory Alert','Supplier Alternative Supplier 1', 'Supplier Alternative Supplier Qty:', 'Supplier Alternative Supplier 2', 'Supplier Alternative Supplier 3','Supplier Product Line',
'Product Family','Product Material Family', 'Material Description', 'Commodity Group', 'Plant', 'Responsibility Area',
'Deere Location Name', 'Deere Alt Location 1 Name', 'Deere Alt Location 2 Name', 'Deere Stock', 'Transit Stock','Country Code',
'Supplier #', 'Supplier Name', 'Deere Product Family', 'On Hand Iowa', 'On Hand Tenessee', 'Confirmed Qty', 'Minimum Shipping Date',
'Tier N Supplier Location', 'In Transit Inventory to Supplier', 'PO Due Date', 'Actual Deere PO Demand', 'Forecasted Deere PO Demand',
'Country', 'State', 'City', 'JDDemand_6mos', 'JDDemand_12mos', 'Cumulative PO demand', 'LeadTime']

In [6]:
# columns = ['PO Number', 'Part Number', 'SAP Line Down Date', 'SAP Line Down Including ASN', 'Part on Waterloo Critical Report?', 'Waterloo Report DOH', '# of Units that Use this Part',
# 'Annual EAU per part # (All units)', 'On Hand Inventory at Deere', 'In Transit Inventory to Deere', 'Deere Ordering Location Stock + ASN Qty',
# 'Supplier Stock', 'P/N: Per Unit Price (Deere Data/Supplier Data)', 'Base Commodity (Provided by Supplier)', 'Base Commodity Price (3rd Party Data)',
# 'Base Commodity Price Change (3rd Party Data)', 'Base Commodity Change % (3rd Party Data)', 'Supplier Critical Part Number',
# 'Supplier Can Build Critical Part Qty: On Hand', 'Tier N Supplier PO#', 'Tier N Supplier PO Due Date', 'Tier N Supplier PO Forecast Date',
# 'Tier N PO Qty', 'Tier N Supplier Location (Country)', 'Supplier Tier N Supplier Name', 'Critical Part Alert', 'Deere PO Demand',
# 'Is Supplier Tier N Forecast Date within 30 days of Deere Need Date? (0 - No, 1 - Yes)', 'Does Inventory On-Hand Meet Demand including able to build (0 - No, 1 - Yes)',
# 'Inventory Alert','Supplier Alternative Supplier 1', 'Supplier Alternative Supplier Qty:', 'Supplier Alternative Supplier 2', 'Supplier Alternative Supplier 3','Supplier Product Line',
# 'Product Family','Product Material Family', 'Material Description', 'Commodity Group', 'Plant', 'Responsibility Area',
# 'Deere Location Name', 'Deere Alt Location 1 Name', 'Deere Alt Location 2 Name', 'Deere Stock', 'Transit Stock','Country Code',
# 'Supplier #', 'Supplier Name', 'Deere Product Family', 'On Hand Iowa', 'On Hand Tenessee', 'Confirmed Qty', 'Minimum Shipping Date',
# 'Tier N Supplier Location', 'In Transit Inventory to Supplier', 'PO Due Date', 'Actual Deere PO Demand', 'Forecasted Deere PO Demand',
# 'Country', 'State', 'City', 'JDDemand_6mos', 'JDDemand_12mos', 'Cumulative PO demand', 'LeadTime']

In [7]:
# Update the mapping as new location comes in
bossard['Tier N Supplier Location (Country)'] = bossard['VendorCountry'].replace({"CH": "Switzerland", "TW": "Taiwan", "IN":"India",
                                                                       "US": "United States", "CN": "China", "DE": "Germany",
                                                                       "BR": "Brazil", "KR": "Korea", "CA": "Canada", "TR":"Turkey",
                                                                       "IT": "Italy", "VN": "Vietnam", "GB": "Great Britain"})
bossard = bossard.rename(columns = {"JDPartNumber":"Part Number"})

In [8]:
# deere_processed = deere.dropna(subset = ['SAP Line Down Date', 'PO Due Date', 'PO Placement Date', 'SAP Line Down Including ASN'])
deere_processed = deere.copy()
deere_processed['Part Number'] = deere['Part Number'].str.strip()

In [9]:
deere_part_number = set(deere_processed['Part Number'])
bossard_part_number = set(bossard['Part Number'])

In [10]:
common_parts = deere_part_number.intersection(bossard_part_number)
deere_not_bossard = deere_part_number - bossard_part_number
bossard_not_deere = bossard_part_number - deere_part_number

In [11]:
d = {"Common Part Number": list(common_parts), 
     "Parts in Deere but not is Bossard": list(deere_not_bossard),
     "Parts in Bossard but not is Deere": list(bossard_not_deere)}

missing_parts = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in d.items() ]))

# Bossard Final write
missing_parts.to_csv('Parts_Mismatch.csv', index = False)

prefix = 'Final/bossard/'
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'Parts_Mismatch.csv')).upload_file('Parts_Mismatch.csv')

In [12]:
# len(deere_part_number), len(bossard_part_number), len(common_parts), len(deere_not_bossard), len(bossard_not_deere)

In [13]:
merge_file = pd.merge(deere_processed, bossard, on = "Part Number", how = "inner")

In [14]:
len(merge_file.columns)

40

In [15]:
cols_to_del = ['Part Number.1', 'Deere Ordering Plant Code.1', 'Part Number Plant Code', 'Deere Product Type',
               'OnShipping_MinETA', 'TotalPipeline', 'VendorCountry']

merge_file = merge_file.drop(cols_to_del, axis = 1)

In [16]:
merge_file['PO Number'] = merge_file['PO Number'].astype(str)
merge_file['SAP Line Down Including ASN'] = pd.to_datetime(merge_file['SAP Line Down Including ASN'], errors='coerce')
merge_file['Confirmed_MinShipDt'] = pd.to_datetime(merge_file['Confirmed_MinShipDt'])
merge_file['SAP Line Down Date'] = pd.to_datetime(merge_file['SAP Line Down Date'])
merge_file['PO Placement Date'] = pd.to_datetime(merge_file['PO Placement Date']).dt.date #### Issue
merge_file['PO Placement Date'] = pd.to_datetime(merge_file['PO Placement Date'])
merge_file['PO Due Date'] = pd.to_datetime(merge_file['PO Due Date'])
merge_file['Waterloo Report DOH'] = merge_file['Waterloo Report DOH'].astype(float)
merge_file['Annual EAU per part # (All units)'] = merge_file['Annual EAU per part # (All units)'].astype(int)
merge_file['Deere PO Demand'] = merge_file['Deere PO Demand'].astype(int)
merge_file['Deere Ordering Location ASN Qty'] = merge_file['Deere Ordering Location ASN Qty'].astype(int)
merge_file['Deere Ordering Location Stock + ASN Qty'] = merge_file['Deere Ordering Location Stock + ASN Qty'].astype(int) #
merge_file['Deere Responsibility Area'] = merge_file['Deere Ordering Plant Code'].astype(str)
merge_file['Responsibility Area'] = merge_file['Deere Ordering Plant Code'].astype(str)
merge_file['Supplier #'] = merge_file['Supplier #'].astype(int)
merge_file['OnHandTN'] = merge_file['OnHandTN'].astype(float).astype(int)
merge_file['OnHandIA'] = merge_file['OnHandIA'].astype(float).astype(int)
merge_file['qty_OnShipping'] = merge_file['qty_OnShipping'].astype(float).astype(int)
merge_file['qty_Confirmed'] = merge_file['qty_Confirmed'].astype(float).astype(int)

In [17]:
str_cols = merge_file.columns[merge_file.dtypes == 'object']
for col in str_cols:
    merge_file[col] = merge_file[col].str.strip()

In [18]:
deere_country_map = {'Harvester Works - East Moline (US)': 'United States', 'Waterloo Works (US)': 'United States',
                    'Dubuque Works (US)': 'United States', 'Horicon Works (US)': 'United States', 'Davenport Works (US)':'United States',
                    'Des Moines Works (US)':'United States', 'Thibodaux (US)':'United States', 'Seeding - Moline (US)':'United States',
                    'Augusta / Commercial Products (US)': 'United States', 'Greeneville / Power Products (US)': 'United States',
                    'Seeding - Valley City (US)': 'United States', 'Turf Care - Fuquay-Varina (US)': 'United States',
                    'PATON (US)':'United States', 'Ottumwa Works (US)':'United States', 'Cylinder Division - Moline (US)': 'United States',
                    'WORLDWIDE LOGISTICS  (US)': 'United States', 'SUNBELT - CHARLOTTE (US)': 'United States', 'Motores Torreon (Mexico)': 'Mexico',
                    'JD Reman - Springfield (US)': 'United States', 'Coffeyville Works (US)': 'United States',
                    'Engine Works - Waterloo (US)': 'United States'}

merge_file['Country'] = merge_file['Deere Ordering Location Name'].replace(deere_country_map)

In [19]:
deere_state_map = {'Harvester Works - East Moline (US)': 'Illinois', 'Waterloo Works (US)': 'Iowa',
                    'Dubuque Works (US)': 'Iowa', 'Horicon Works (US)': 'Wisconsin', 'Davenport Works (US)':'Iowa',
                    'Des Moines Works (US)':'Iowa', 'Thibodaux (US)':'Louisiana', 'Seeding - Moline (US)':'Illinois',
                    'Augusta / Commercial Products (US)': 'Georgia', 'Greeneville / Power Products (US)': 'Tennessee',
                    'Seeding - Valley City (US)': 'North Dakota', 'Turf Care - Fuquay-Varina (US)': 'North Carolina',
                    'PATON (US)':'Iowa', 'Ottumwa Works (US)':'Iowa', 'Cylinder Division - Moline (US)': 'Illinois',
                    'WORLDWIDE LOGISTICS  (US)': 'Illinois', 'SUNBELT - CHARLOTTE (US)': 'North Carolina', 'Motores Torreon (Mexico)': 'NA',
                    'JD Reman - Springfield (US)': 'Missouri', 'Coffeyville Works (US)': 'Kansas', 'Engine Works - Waterloo (US)': 'Iowa',}
merge_file['State'] = merge_file['Deere Ordering Location Name'].replace(deere_state_map)

In [20]:
deere_city_map = {'Harvester Works - East Moline (US)': 'East Moline', 'Waterloo Works (US)': 'Waterloo',
                    'Dubuque Works (US)': 'Dubuque', 'Horicon Works (US)': 'Horicon', 'Davenport Works (US)':'Davenport',
                    'Des Moines Works (US)':'Des Moines', 'Thibodaux (US)':'Thibodaux', 'Seeding - Moline (US)':'Moline',
                    'Augusta / Commercial Products (US)': 'Augusta', 'Greeneville / Power Products (US)': 'Greeneville',
                    'Seeding - Valley City (US)': 'Valley City', 'Turf Care - Fuquay-Varina (US)': 'Fuquay-Varina',
                    'PATON (US)':'Paton', 'Ottumwa Works (US)':'Ottumwa', 'Cylinder Division - Moline (US)': 'Moline',
                    'WORLDWIDE LOGISTICS  (US)': 'Moline', 'SUNBELT - CHARLOTTE (US)': 'Charlotte', 'Motores Torreon (Mexico)': 'Torreon', 
                    'JD Reman - Springfield (US)': 'Springfield', 'Coffeyville Works (US)': 'Coffeyville', 'Engine Works - Waterloo (US)': 'Waterloo',
                 }
merge_file['City'] = merge_file['Deere Ordering Location Name'].replace(deere_city_map)

In [21]:
# Calculated Fields
merge_file['On Hand Inventory at Deere'] = merge_file['Deere Ordering Location Stock + ASN Qty'] - merge_file['Deere Ordering Location ASN Qty']
merge_file['Critical Part Alert'] = merge_file['Part on Waterloo Critical Report?']
merge_file['Deere Stock'] = merge_file['On Hand Inventory at Deere']
merge_file['Transit Stock'] = merge_file['Deere Ordering Location ASN Qty']
merge_file['In Transit Inventory to Deere'] = merge_file['Deere Ordering Location ASN Qty']
merge_file['Base Commodity'] = merge_file['MaterialDesc']

# Mock Data

In [22]:
def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)


n_tier_po_due_date = []
for dates in range(merge_file.shape[0]):
    d1 = datetime.datetime.strptime('1/1/2015', '%m/%d/%Y')
    d2 = datetime.datetime.strptime('1/1/2021', '%m/%d/%Y')

    n_tier_po_due_date.append(random_date(d1, d2))
    
n_tier_po_forecast_date = []
for dates in range(merge_file.shape[0]):
    d1 = datetime.datetime.strptime('1/1/2015', '%m/%d/%Y')
    d2 = datetime.datetime.strptime('1/1/2021', '%m/%d/%Y')

    n_tier_po_forecast_date.append(random_date(d1, d2))

In [23]:
merge_file['Supplier Stock'] = [np.random.randint(0,10000) for i in range(merge_file.shape[0])]
merge_file['P/N: Per Unit Price (Deere Data/Supplier Data)'] = [np.random.randint(500,1000) for i in range(merge_file.shape[0])]
# merge_file['Base Commodity Price (3rd Party Data)'] = [np.random.randint(35,9000) for i in range(merge_file.shape[0])]
# merge_file['Base Commodity Price Change (3rd Party Data)'] = np.random.choice(a = ['increase', 'decrease'], size = merge_file.shape[0],
#                                                                                p = [0.6, 0.4])
# merge_file['Base Commodity Change % (3rd Party Data)'] = [np.random.randint(0.01,2.5) for i in range(merge_file.shape[0])]
merge_file['Supplier Critical Part Number'] = [''.join(random.choices(string.ascii_letters + string.digits, k=10)) for i in range(merge_file.shape[0])]
merge_file['Supplier Can Build Critical Part Qty: On Hand'] = [np.random.randint(2,35800) for i in range(merge_file.shape[0])]
merge_file['Tier N Supplier PO#'] = [str(np.random.randint(5000000000,6000000000)) for i in range(merge_file.shape[0])]
merge_file['Tier N PO Qty'] = [np.random.randint(0,10000) for i in range(merge_file.shape[0])]
# merge_file['Tier N Supplier Location (Country)'] = bossard['Tier_N_Supplier_Location']# merge_file['Country']
merge_file['Supplier Tier N Supplier Name'] = np.random.choice(a = ['A', 'B', 'C', 'D', 'E'], size = merge_file.shape[0],
                                                                               p = [0.1, 0.2, 0.3, 0.3, 0.1])
merge_file['Is Supplier Tier N Forecast Date within 30 days of Deere Need Date? (0 - No, 1 - Yes)'] = np.random.choice(a = ['No', 'Yes'], size = merge_file.shape[0],
                                                                               p = [0.6, 0.4])
merge_file['Does Inventory On-Hand Meet Demand including able to build (0 - No, 1 - Yes)'] = np.random.choice(a = ['No', 'Yes'], size = merge_file.shape[0],
                                                                               p = [0.6, 0.4])
merge_file['Material Description'] = merge_file['Deere Material Description']
merge_file['Commodity Group'] = merge_file['Deere Commodity Group']
merge_file['Plant'] = merge_file['Deere Ordering Plant Code']
merge_file['Deere Responsibility Area'] = merge_file['Deere Responsibility Area'].astype(str)
# merge_file['Responsibility Area'] = merge_file['Deere Responsibility Area']
merge_file['Deere Location Name'] = merge_file['Deere Ordering Location Name']
merge_file['Tier N Supplier PO Due Date'] = n_tier_po_due_date
merge_file['Tier N Supplier PO Forecast Date'] = n_tier_po_forecast_date
merge_file['Product Family'] = merge_file['Deere Product Family']
merge_file['Inventory Alert'] = np.random.choice(a = ['Yes', 'No'], size = merge_file.shape[0],
                                                                               p = [0.9, 0.1])
merge_file['Deere Alt Location 1 Name'] = merge_file['Deere Ordering Location Name']
merge_file['Country Code'] = merge_file['Bossard Locations']
merge_file['On Hand Iowa'] = merge_file['OnHandIA']
merge_file['On Hand Tenessee'] = merge_file['OnHandTN']
merge_file['Confirmed Qty'] = merge_file['qty_Confirmed']
merge_file['Minimum Shipping Date'] = merge_file['Confirmed_MinShipDt']
# merge_file['Tier N Supplier Location (Country)'] = merge_file['Country']
merge_file['In Transit Inventory to Supplier'] = merge_file['qty_OnShipping']

# mask1 = merge_file['PO Due Date']  < date.today().strftime("%Y/%m/%d")
mask1 = merge_file['PO Due Date'] <= "2021-11-30"#"30-11-2021"
# mask2 = merge_file['PO Due Date']  >= (date.today() - datetime.timedelta(days=30)).strftime("%Y/%m/%d")
mask2 = merge_file['PO Due Date'] >= "2021-11-01"#"1-11-2021"
merge_file['Actual Deere PO Demand'] = merge_file['Deere PO Demand'].where(mask1)
merge_file['Forecasted Deere PO Demand'] = merge_file['Deere PO Demand'].where(mask2)
merge_file['Supplier Alternative Supplier 1'] = 'Deere & Co.'
merge_file['Supplier Alternative Supplier Qty:'] = [np.random.randint(2,35800) for i in range(merge_file.shape[0])]
merge_file['Supplier Alternative Supplier 2'] = 'MOCK DATA'
merge_file['Supplier Alternative Supplier 3'] = 'MOCK DATA'
merge_file['Supplier Product Line'] = 'MOCK DATA'
merge_file['Product Material Family'] = 'MOCK DATA'
merge_file['Deere Alt Location 2 Name'] = 'MOCK DATA'
merge_file['Tier N Supplier Location'] = merge_file['Country']

In [24]:
merge_file_subset = merge_file[['PO Number', 'SAP Line Down Date', 'Deere PO Demand', 'PO Due Date']]
merge_file_subset = merge_file_subset.fillna(0)
merge_file_subset.head()

,PO Number,SAP Line Down Date,Deere PO Demand,PO Due Date
0,5501039548,0,350,2022-01-10
1,5501039548,0,350,2022-02-03
2,5501039548,0,350,2022-01-20
3,5501039548,0,350,2022-02-24
4,5501039548,0,350,2022-02-14


In [25]:
cum_po_demand = []
for count, po in enumerate (merge_file_subset['PO Number'].unique()):
    temp_df = merge_file_subset[merge_file_subset['PO Number'] == po].reset_index(drop = True)
    line_down_date = temp_df['SAP Line Down Date'][0]
    if line_down_date == 0:
        temp_df['Cumulative PO demand'] = np.nan
    else:
        temp_df_subset = temp_df[temp_df['PO Due Date'] <= line_down_date]
        temp_df['Cumulative PO demand'] = temp_df_subset['Deere PO Demand'].sum()
    cum_po_demand.append(temp_df)

In [26]:
cum_po_demand_df = pd.concat(cum_po_demand, axis = 0).drop(['SAP Line Down Date', 'Deere PO Demand', 'PO Due Date'], axis = 1).drop_duplicates()

In [27]:
merge_file = pd.merge(cum_po_demand_df, merge_file, on = "PO Number", how = 'inner')

In [28]:
merge_file = pd.merge(merge_file, commodity, on = 'Base Commodity', how = 'left')

In [29]:
# Filter the required columns
merge_file = merge_file[columns]

In [30]:
merge_file = merge_file.rename(columns = {"JDDemand_6mos": "Deere EDI 6 months", "JDDemand_12mos": "Deere EDI 12 months",
                                         "Annual EAU per part # (All units)": 'Estimated Annual Usage per piece'})

In [31]:
merge_file.shape

(409758, 66)

In [32]:
merge_file.head()

,PO Number,Part Number,SAP Line Down Date,SAP Line Down Including ASN,Part on Waterloo Critical Report?,Waterloo Report DOH,# of Units that Use this Part,Estimated Annual Usage per piece,On Hand Inventory at Deere,In Transit Inventory to Deere,Deere Ordering Location Stock + ASN Qty,Supplier Stock,P/N: Per Unit Price (Deere Data/Supplier Data),Base Commodity,Base Commodity PPI,Base Commodity PPI Change,Base Commodity Change%,Supplier Critical Part Number,Supplier Can Build Critical Part Qty: On Hand,Tier N Supplier PO#,Tier N Supplier PO Due Date,Tier N Supplier PO Forecast Date,Tier N PO Qty,Tier N Supplier Location (Country),Supplier Tier N Supplier Name,Critical Part Alert,Deere PO Demand,"Is Supplier Tier N Forecast Date within 30 days of Deere Need Date? (0 - No, 1 - Yes)","Does Inventory On-Hand Meet Demand including able to build (0 - No, 1 - Yes)",Inventory Alert,Supplier Alternative Supplier 1,Supplier Alternative Supplier Qty:,Supplier Alternative Supplier 2,Supplier Alternative Supplier 3,Supplier Product Line,Product Family,Product Material Family,Material Description,Commodity Group,Plant,Responsibility Area,Deere Location Name,Deere Alt Location 1 Name,Deere Alt Location 2 Name,Deere Stock,Transit Stock,Country Code,Supplier #,Supplier Name,Deere Product Family,On Hand Iowa,On Hand Tenessee,Confirmed Qty,Minimum Shipping Date,Tier N Supplier Location,In Transit Inventory to Supplier,PO Due Date,Actual Deere PO Demand,Forecasted Deere PO Demand,Country,State,City,Deere EDI 6 months,Deere EDI 12 months,Cumulative PO demand,LeadTime
0,5501039548,19M9090,NaT,NaT,No,0.0,NaN,39900,0,0,0,2714,582,Steel,418.0,Increase,3.747828,vJKC3J1xqy,17318,5785941394,2020-07-18 07:21:00,2016-09-23 18:09:10,4303,Brazil,D,No,350,No,Yes,Yes,Deere & Co.,25175,MOCK DATA,MOCK DATA,MOCK DATA,NaN,MOCK DATA,CAP SCREW,Aftermarket Ferrous & Tracks,LV00,LV00,Augusta / Commercial Products (US),Augusta / Commercial Products (US),MOCK DATA,0,0,US,2666,BOSSARD INC.,NaN,0,9647,0,NaT,United States,19424,2022-01-10,NaN,350.0,United States,Georgia,Augusta,18900.0000000000,39200.0000000000,NaN,32
1,5501039548,19M9090,NaT,NaT,No,0.0,NaN,39900,0,0,0,1148,606,Steel,418.0,Increase,3.747828,WJ7dFpvInf,2692,5879570999,2020-09-29 03:20:41,2016-10-28 16:25:37,3274,Brazil,B,No,350,Yes,Yes,Yes,Deere & Co.,26751,MOCK DATA,MOCK DATA,MOCK DATA,NaN,MOCK DATA,CAP SCREW,Aftermarket Ferrous & Tracks,LV00,LV00,Augusta / Commercial Products (US),Augusta / Commercial Products (US),MOCK DATA,0,0,US,2666,BOSSARD INC.,NaN,0,9647,0,NaT,United States,19424,2022-02-03,NaN,350.0,United States,Georgia,Augusta,18900.0000000000,39200.0000000000,NaN,32
2,5501039548,19M9090,NaT,NaT,No,0.0,NaN,39900,0,0,0,8328,838,Steel,418.0,Increase,3.747828,XDgDksxEBJ,22584,5861532184,2020-06-19 10:33:46,2017-12-14 07:02:34,5950,Brazil,B,No,350,Yes,No,No,Deere & Co.,15153,MOCK DATA,MOCK DATA,MOCK DATA,NaN,MOCK DATA,CAP SCREW,Aftermarket Ferrous & Tracks,LV00,LV00,Augusta / Commercial Products (US),Augusta / Commercial Products (US),MOCK DATA,0,0,US,2666,BOSSARD INC.,NaN,0,9647,0,NaT,United States,19424,2022-01-20,NaN,350.0,United States,Georgia,Augusta,18900.0000000000,39200.0000000000,NaN,32
3,5501039548,19M9090,NaT,NaT,No,0.0,NaN,39900,0,0,0,76,618,Steel,418.0,Increase,3.747828,ezSBXmkser,19305,5273184558,2015-01-31 03:13:13,2020-03-05 08:21:01,3772,Brazil,E,No,350,No,Yes,No,Deere & Co.,33742,MOCK DATA,MOCK DATA,MOCK DATA,NaN,MOCK DATA,CAP SCREW,Aftermarket Ferrous & Tracks,LV00,LV00,Augusta / Commercial Products (US),Augusta / Commercial Products (US),MOCK DATA,0,0,US,2666,BOSSARD INC.,NaN,0,9647,0,NaT,United States,19424,2022-02-24,NaN,350.0,United States,Georgia,Augusta,18900.0000000000,39200.0000000000,NaN,32
4,5501039548,19M9090,NaT,NaT,No,0.0,NaN,39900,0,0,0,8443,508,Steel,418.0,Increase,3.747828,LRScJSo9f9,18411,5954679640,2019-04-18 07:00:59,2018-08-28 19:58:23,7638,Brazil,C,No,350,No,No,Yes,Deere & Co.,17806,MOCK DATA,MOCK DATA,MOCK DATA,NaN,MOCK DATA,CAP SCREW,Aftermarket Ferrous & Tracks

In [33]:
# Bossard Final write
merge_file.to_csv('Bossard Live Main Table.csv', index = False)

prefix = 'Final/bossard/Bossard Live Main Table/' 
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'Bossard Live Main Table.csv')).upload_file('Bossard Live Main Table.csv')

# Bossard Inventory

In [34]:
upper = merge_file[['PO Number', 'Part Number', 'On Hand Iowa']]
upper['Location'] = 'Iowa'
upper['City'] = 'Cedar Falls'

lower = merge_file[['PO Number', 'Part Number', 'On Hand Tenessee']]
lower['Location'] = 'Tennessee'
lower['City'] = 'Greeneville'
lower = lower.rename(columns = {'On Hand Tenessee': 'On Hand Iowa'})

bossard_inventory = pd.concat([upper, lower], axis = 0)
bossard_inventory = bossard_inventory.rename(columns = {'On Hand Iowa': 'On Hand'})

In [35]:
# Bossard Final write
bossard_inventory.to_csv('Bossard Live Inventory.csv', index = False)

prefix = 'Final/bossard/Bossard Live Inventory/'
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'Bossard Live Inventory.csv')).upload_file('Bossard Live Inventory.csv')

# Bossard Alternative Supplier

In [36]:
alternate_supplier = merge_file[['PO Number', 'Part Number', 'Deere Location Name']]
alternate_supplier['Deere Ordering Location Stock'] = merge_file['On Hand Inventory at Deere']
alternate_supplier = alternate_supplier.drop_duplicates()
alternate_supplier = alternate_supplier[['PO Number', 'Part Number', 'Deere Ordering Location Stock', 'Deere Location Name']]

In [37]:
def inner_func(data_part):
    temp_list = []
    if len(data_part) == 1:
        data_part['Category'] = 'Ordering Plant'
        return data_part
    else:
        for i in range(len(data_part)):
            temp1 = data_part.iloc[[i]]
            temp1['Category'] = 'Ordering Plant'
            temp2 = data_part.drop(i)
            temp2['Category'] = "Alternative Supplier"
            temp2['PO Number'] = temp1['PO Number'].tolist()[0]
            temp2['label'] = list(range(len(temp2)))
            temp2['Category'] = temp2['Category'].astype(str) + " " + temp2['label'].astype(str)

            temp3 = pd.concat([temp1, temp2], axis = 0)
            temp3 = temp3.drop_duplicates(subset = 'Deere Location Name', keep="first")
            temp_list.append(temp3)
    
        return pd.concat(temp_list, axis = 0).drop('label', axis = 1)

In [38]:
result = []
for pn in alternate_supplier['Part Number'].unique():
    data_part = alternate_supplier[alternate_supplier["Part Number"] == pn].drop_duplicates().reset_index(drop = True)
    result.append(inner_func(data_part))

In [39]:
final_result = pd.concat(result, axis = 0)

In [40]:
final_result_first = final_result.groupby(['PO Number', 'Part Number']).head(1)

In [41]:
dfs= []
po_on_combo = final_result[['PO Number', 'Part Number']].drop_duplicates().values
for i in range(len(po_on_combo)):
#     print(i)
    po = po_on_combo[i][0]
    pn = po_on_combo[i][1]
    temp_df = final_result[(final_result['Part Number'] == pn) & (final_result['PO Number'] == po)]
    if temp_df.shape[0] == 1:
#         print("Single row for PO Part combination")
        continue
    else:
        temp_df = temp_df[1:]
        temp_df = temp_df.sort_values(by = 'Deere Ordering Location Stock', ascending = False)
    temp_df['New Category'] = list(range(1, len(temp_df) + 1))
    temp_df['Category'] = 'Alternate Supplier ' + temp_df['New Category'].astype(str)
    temp_df = temp_df.drop('New Category', axis = 1)
    dfs.append(temp_df)

sorted_df = pd.concat(dfs, axis = 0)

In [42]:
location_table = pd.concat([final_result_first, sorted_df], axis = 0)

In [43]:
location_table['Country'] = location_table['Deere Location Name'].replace(deere_country_map)
location_table['State'] = location_table['Deere Location Name'].replace(deere_state_map)
location_table['City'] = location_table['Deere Location Name'].replace(deere_city_map)

In [44]:
add = []
for i in range(len(po_on_combo)):
    po = po_on_combo[i][0]
    pn = po_on_combo[i][1]
    temp_df = location_table[(location_table['Part Number'] == pn) & (location_table['PO Number'] == po)]
    temp_df['On Hand Inventory at Deere'] = temp_df['Deere Ordering Location Stock'].sum()
    add.append(temp_df)
    
location_table = pd.concat(add, axis = 0)

In [45]:
merge_file_subset = merge_file[['PO Number', 'Part Number', 'In Transit Inventory to Supplier']]
merge_file_subset = merge_file_subset.groupby(['PO Number', 'Part Number']).max().reset_index()

In [46]:
transit_inv_sum = []
for i in range(len(po_on_combo)):
#     print(i)
    po = po_on_combo[i][0]
    pn = po_on_combo[i][1]
    merge_temp_df = merge_file_subset[(merge_file_subset['Part Number'] == pn) & (merge_file_subset['PO Number'] == po)]['In Transit Inventory to Supplier'].values[0]
    temp_df = location_table[(location_table['Part Number'] == pn) & (location_table['PO Number'] == po)]
    temp_df['In Transit Inventory to Supplier'] = merge_temp_df
    transit_inv_sum.append(temp_df)

location_table = pd.concat(transit_inv_sum, axis = 0)

In [47]:
merge_file_deer_inv = merge_file[['PO Number', 'Part Number', 'In Transit Inventory to Deere']]
merge_file_deer_inv = merge_file_deer_inv.groupby(['PO Number', 'Part Number']).sum().reset_index()

In [48]:
transit_inv_deere_sum = []
for i in range(len(po_on_combo)):
#     print(i)
    po = po_on_combo[i][0]
    pn = po_on_combo[i][1]
    merge_temp_df = merge_file_deer_inv[(merge_file_deer_inv['Part Number'] == pn) & (merge_file_deer_inv['PO Number'] == po)]['In Transit Inventory to Deere'].values[0]
    temp_df = location_table[(location_table['Part Number'] == pn) & (location_table['PO Number'] == po)]
    temp_df['In Transit Inventory to Deere'] = merge_temp_df
    transit_inv_deere_sum.append(temp_df)

location_table = pd.concat(transit_inv_deere_sum, axis = 0)

In [49]:
merge_file_ia_tn = merge_file[['PO Number', 'Part Number']]
merge_file_ia_tn['On Hand Inventory at Supplier'] = merge_file['On Hand Iowa'] + merge_file['On Hand Tenessee']
merge_file_ia_tn = merge_file_ia_tn.groupby(['PO Number', 'Part Number']).max().reset_index()

In [50]:
transit_io_tn_sum = []
for i in range(len(po_on_combo)):
#     print(i)
    po = po_on_combo[i][0]
    pn = po_on_combo[i][1]
    merge_temp_df = merge_file_ia_tn[(merge_file_ia_tn['Part Number'] == pn) & (merge_file_ia_tn['PO Number'] == po)]['On Hand Inventory at Supplier'].values[0]
    temp_df = location_table[(location_table['Part Number'] == pn) & (location_table['PO Number'] == po)]
    temp_df['On Hand Inventory at Supplier'] = merge_temp_df
    transit_io_tn_sum.append(temp_df)

location_table = pd.concat(transit_io_tn_sum, axis = 0)

In [51]:
# Bossard Final write
location_table.to_csv('Bossard Live Location Table.csv', index = False)
    
prefix = 'Final/bossard/Bossard Live Location Table/'
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'Bossard Live Location Table.csv')).upload_file('Bossard Live Location Table.csv')

# Time Stamp

In [52]:
from datetime import date
from datetime import datetime
from pytz import timezone

now = datetime.now(timezone('US/Central')) # 'America/Chicago'
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
# today = date.today()
# today = today.strftime("%d/%m/%Y")
date_df = pd.DataFrame({'Last Updated (IST)': [dt_string]})
date_df.columns = ['Last Updated (CST)']
time = date_df['Last Updated (CST)'][0][-9:]
date_df['Last Updated (CST)'] = pd.to_datetime(date_df['Last Updated (CST)'])
date_df['Last Updated (CST)'] = date_df['Last Updated (CST)'].dt.strftime('%d/%m/%Y')
date_df['Last Updated (CST)'] = date_df['Last Updated (CST)'].astype(str) + time

# Bossard Final write
date_df.to_csv('Bossard Live Timestamp.csv', index = False)
    
prefix = 'Final/bossard/BossardTimestamp/'
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'Bossard Live Timestamp.csv')).upload_file('Bossard Live Timestamp.csv')

In [53]:
print("Code ran successfully!!")

Code ran successfully!!
